In [1]:
import cobra
from cobra.io import load_model, read_sbml_model
from cobra.flux_analysis import flux_variability_analysis
import pheflux
import pandas as pd
import scipy
from scipy.stats import pearsonr
import numpy as np
import matplotlib.pyplot as plt
import importlib
import gc
from cobra.sampling import sample
from matplotlib.ticker import MaxNLocator

Welcome to PheFlux ! 



/usr/local/lib/python3.10/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
%pwd

'/home/marcelo/jupyter/teraflux/fig5/Sstipitis'

In [21]:
import sys
teraflux_path = '../../'
if teraflux_path not in sys.path:
    sys.path.append(teraflux_path)
import teraflux

# También puedes usar la función de autoreload para que Jupyter
# recargue tu librería si haces cambios en el archivo .py
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
organism = "Sstipitis"
model_dir = "./gems/iTL885.xml"
model = read_sbml_model(model_dir)
solution=model.optimize()
objective_id=[rxn.id for rxn in model.summary()._objective.keys()][0]

medium_codes   = ["glc","glc_chem"]
reaction_codes = {"glc":["SS1232"],"glc_chem":["SS1232"],"Growth_rate":[objective_id]}

In [13]:
# Compute max magnitude for the standard bounds set
max_mag = max(
    max(abs(rxn.lower_bound), abs(rxn.upper_bound))
    for rxn in model.reactions
)

# Define standard bounds
STANDARD_BOUNDS = {0.0, max_mag, -max_mag}

for reaction in model.reactions:
    lb = reaction.lower_bound
    ub = reaction.upper_bound
    if lb==0 and ub==0: print(reaction.id,(lb,ub),reaction.bounds,sep="\t")
    # Only process non-standard bounds
    if lb not in STANDARD_BOUNDS or ub not in STANDARD_BOUNDS:

        # Category 1: always positive
        if lb >= 0 and ub > 0:
            reaction.bounds=(0,max_mag)
        # Category 2: always negative
        elif ub <= 0 and lb < 0:
            reaction.bounds=(-max_mag,0)
        # Category 3: crossing zero
        elif lb < 0 < ub:
            reaction.bounds=(-max_mag,max_mag)
        print(reaction.id,(lb,ub),reaction.bounds,sep="\t")

SS0209	(0.0, 0.0)	(0.0, 0.0)
SS0700	(2.6, 2.6)	(0, 1000.0)
SS0943	(0.0, 0.0)	(0.0, 0.0)
SS1232	(-100.0, 1000.0)	(-1000.0, 1000.0)


In [20]:
model.reactions.SS0209

Reaction identifier,SS0209
Name,
Memory address,0x7f5505d7f5e0
Stoichiometry,ETH_c + NADP_c --> ACAL_c + H_c + NADPH_c ETH[c] + NADP[c] --> ACAL[c] + H[c] + NADPH[c]
GPR,PICST_27980
Lower bound,0.0
Upper bound,0.0


In [17]:
# Create medium file
for medium_code in medium_codes:
    model = read_sbml_model(model_dir)
    # Define carbon source
    model.reactions.SS1232.bounds=(0,max_mag) 
    for rxn in reaction_codes[medium_code]:
        model.reactions.get_by_id(rxn).bounds=(-4.45,max_mag)#(-5.13,max_mag)
    solution=model.optimize()
    print(medium_code)
    print(model.summary())
    with open(f"mediums/{organism}_Medium_{medium_code}.csv", 'w', newline="\n") as f:
        print("Metabolite_Name", "Reaction_ID", sep="\t", file=f)
        for index,row in model.summary().uptake_flux.iterrows():
            print(row.metabolite, row.reaction, sep="\t", file=f)

glc
Objective
1.0 SS1240 = 0.2906107471028845

Uptake
------
Metabolite Reaction    Flux  C-Number C-Flux
     SLF_e   SS1225 0.03691         0  0.00%
     NH3_e   SS1230   1.869         0  0.00%
      PI_e   SS1231 0.07511         0  0.00%
     GLC_e   SS1232    4.45         0  0.00%
      O2_e   SS1233   15.55         0  0.00%

Secretion
---------
Metabolite Reaction       Flux  C-Number C-Flux
  C60COA_e   SS1114 -0.0006105         0  0.00%
     CO2_e   SS1142     -15.94         0  0.00%
       H_e   SS1169     -10.67         0  0.00%
     H2O_e   SS1229     -19.79         0  0.00%

glc_chem
Objective
1.0 SS1240 = 0.2906107471028845

Uptake
------
Metabolite Reaction    Flux  C-Number C-Flux
     SLF_e   SS1225 0.03691         0  0.00%
     NH3_e   SS1230   1.869         0  0.00%
      PI_e   SS1231 0.07511         0  0.00%
     GLC_e   SS1232    4.45         0  0.00%
      O2_e   SS1233   15.55         0  0.00%

Secretion
---------
Metabolite Reaction       Flux  C-Number C-Flux
  

In [7]:
# Filter the known fluxes to just output the carbon source consumption and biomass growth rate
for medium_code in medium_codes:
    
    experimentalFile=f"./experimental/{organism}_Fluxfile_{medium_code}.csv"
    known_fluxes_filename   =f"./knownFluxes/{organism}_knownFluxes_{medium_code}.csv" 
    experimental = pd.read_csv(experimentalFile,index_col=0,sep="\t")
    reactions = reaction_codes[medium_code]+reaction_codes["Growth_rate"] 

    
    with open(known_fluxes_filename, 'w', newline="\n") as f:
        print("Reaction_ID", "Metabolite_ID", "Reaction_Flux", sep="\t", file=f)
        for Reaction_ID in reactions:
            Metabolite_ID = Reaction_ID
            if Reaction_ID == reaction_codes["Growth_rate"][0]:
                Reaction_Flux = experimental.loc["Growth_rate"].Flux 
            else:
                Reaction_Flux = experimental.loc[Reaction_ID].Flux 
            print(Reaction_ID, Metabolite_ID, Reaction_Flux, sep="\t", file=f)

In [8]:
# Run pheflux

for medium_code in medium_codes:
    # Create InputData file
    input_file_name = f"inputData_{organism}_{medium_code}.csv"
    with open(input_file_name, 'w', newline="\n") as f:
        print("Organism\tCondition\tGeneExpFile\tMedium\tNetwork\tKnownFluxes", file=f)    
        Organism = organism
        Condition = medium_code
        GeneExpFile = f"./transcriptomes/{organism}_Expfile_{medium_code}.csv"
        Medium = f"./mediums/{organism}_Medium_{medium_code}.csv"
        Network = model_dir 
        KnownFluxes = f"./knownFluxes/{organism}_knownFluxes_{medium_code}.csv"
        print(Organism, Condition, GeneExpFile, Medium, Network,KnownFluxes, sep="\t", file=f)

    # Save results
    resultsDir = "./results/pheflux/"
    prefix_log_file = f"{organism}_phe_{medium_code}"
    verbosity = True
    fluxes = pheflux.getFluxes(input_file_name, resultsDir, prefix_log_file, verbosity)

[2025/11/27 20:23:27] Condition ejecuted: Sstipitis - glc
[2025/11/27 20:23:27] Loading metabolic model: iTL885
[2025/11/27 20:23:27] Loading transcriptomic data...
[2025/11/27 20:23:27] Updating metabolic model...
[2025/11/27 20:23:27] Running pheflux...
((-4.45*(R_SS1240-R_SS1240_reverse_d82c2))-(0.47*(R_SS1232-R_SS1232_reverse_fe5bf)))


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Total number of variables............................:     1612
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1612
                     variables with only upper bounds:        0
Total number of

In [22]:
# Run teraflux

for medium_code in medium_codes:
    # Create  InputData file
    input_file_name = f"inputData_{organism}_{medium_code}.csv"
    with open(input_file_name, 'w', newline="\n") as f:
        print("Organism\tCondition\tGeneExpFile\tMedium\tNetwork\tKnownFluxes", file=f)    
        Organism = organism
        Condition = medium_code
        GeneExpFile = f"./transcriptomes/{organism}_Expfile_{medium_code}.csv"
        Medium =      f"./mediums/{organism}_Medium_{medium_code}.csv"
        Network = model_dir 
        KnownFluxes = f"./knownFluxes/{organism}_knownFluxes_{medium_code}.csv"
        print(Organism, Condition, GeneExpFile, Medium, Network,KnownFluxes, sep="\t", file=f)

    # Save results
    resultsDir = "./results/teraflux/"
    prefix_log_file = f"{organism}_{medium_code}"
    ipoptParams={
            'print_level': 0, # Suppress solver output for cleaner logs
            'sb': 'yes',       # Suppress IPOPT banner
            'tol': 1e-10,       # Overall tolerance (e.g., 1e-8, 1e-10, etc.)
            'acceptable_tol': 1e-10, # Allowable tolerance before giving up
            'max_iter': 6500,   # Increase maximum iterations if needed
            'nlp_scaling_method' : 'none',
            'constr_viol_tol' : 1e-10,
            'nlp_scaling_method': 'gradient-based',
            'hessian_approximation': 'exact',
        }
    fluxes,fr,lagrange = teraflux.getFluxes(input_file_name, resultsDir, prefix_log_file, ipoptParams)

Found 1 conditions to process.

========================== STARTING: Sstipitis - glc ===========================
[2026/02/06 18:45:06] Loading metabolic model: iTL885.xml
[2026/02/06 18:45:06] Loading transcriptomic data: Sstipitis_Expfile_glc.csv
[2026/02/06 18:45:06] Loading known fluxes: Sstipitis_knownFluxes_glc.csv
[2026/02/06 18:45:06] Updating model (bounds, medium)...
Applying medium from: ./mediums/Sstipitis_Medium_glc.csv
Starting data pre-processing...
Capping FPKM values at 95th percentile: 252.2950
Median 'g' value (E_g) for imputation: 5.2080e+01
Pre-processing finished in 0.00 seconds.
[2026/02/06 18:45:06] Running TeraFlux optimization...
Creating stoichiometric constraints...
Adding known flux constraints...
Solving with standard FBA to get an initial guess (x0)...
Adding known flux constraints...
R_SS1232 -4.45
R_SS1240 0.47
{'R_SS1225': inf, 'R_SS1230': inf, 'R_SS1231': inf, 'R_SS1232': 4.45, 'R_SS1233': inf}
FBA solution found. Objective value: 0.3171. Status: optim

In [24]:
# Run teraflux with all internal fluxes being reversible

for reaction in model.reactions:
    if reaction.id in model.exchanges: continue
    reaction.bounds = (-1000,1000)
cobra.io.write_sbml_model(model,"./gems/iTL885_allReversible.xml")   

for medium_code in medium_codes:
    # Create  InputData file
    input_file_name = f"inputData_{organism}_{medium_code}.csv"
    with open(input_file_name, 'w', newline="\n") as f:
        print("Organism\tCondition\tGeneExpFile\tMedium\tNetwork\tKnownFluxes", file=f)    
        Organism = organism
        Condition = medium_code
        GeneExpFile = f"./transcriptomes/{organism}_Expfile_{medium_code}.csv"
        Medium =      f"./mediums/{organism}_Medium_{medium_code}.csv"
        Network = "./gems/iTL885_allReversible.xml" 
        KnownFluxes = f"./knownFluxes/{organism}_knownFluxes_{medium_code}.csv"
        print(Organism, Condition, GeneExpFile, Medium, Network,KnownFluxes, sep="\t", file=f)

    # Save results
    resultsDir = "./results/teraflux_allReversible/"
    prefix_log_file = f"{organism}_{medium_code}"
    ipoptParams={
            'print_level': 0, # Suppress solver output for cleaner logs
            'sb': 'yes',       # Suppress IPOPT banner
            'tol': 1e-10,       # Overall tolerance (e.g., 1e-8, 1e-10, etc.)
            'acceptable_tol': 1e-10, # Allowable tolerance before giving up
            'max_iter': 6500,   # Increase maximum iterations if needed
            'nlp_scaling_method' : 'none',
            'constr_viol_tol' : 1e-10,
            'nlp_scaling_method': 'gradient-based',
            'hessian_approximation': 'exact',
        }
    fluxes,fr,lagrange = teraflux.getFluxes(input_file_name, resultsDir, prefix_log_file, ipoptParams)

Found 1 conditions to process.

========================== STARTING: Sstipitis - glc ===========================
[2026/02/06 19:02:30] Loading metabolic model: iTL885_allReversible.xml
[2026/02/06 19:02:30] Loading transcriptomic data: Sstipitis_Expfile_glc.csv
[2026/02/06 19:02:30] Loading known fluxes: Sstipitis_knownFluxes_glc.csv
[2026/02/06 19:02:30] Updating model (bounds, medium)...
Applying medium from: ./mediums/Sstipitis_Medium_glc.csv
Starting data pre-processing...
Capping FPKM values at 95th percentile: 252.2950
Median 'g' value (E_g) for imputation: 5.2080e+01
Pre-processing finished in 0.00 seconds.
[2026/02/06 19:02:30] Running TeraFlux optimization...
Creating stoichiometric constraints...
Adding known flux constraints...
Solving with standard FBA to get an initial guess (x0)...
Adding known flux constraints...
R_SS1232 -4.45
R_SS1240 0.47
{'R_SS1225': inf, 'R_SS1230': inf, 'R_SS1231': inf, 'R_SS1232': 4.45, 'R_SS1233': inf}
FBA solution found. Objective value: 0.4700.

In [25]:
%pwd

'/home/marcelo/jupyter/teraflux/fig5/Sstipitis'